## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# loading the dataset
df=pd.read_csv('all_kindle_review.csv')

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
df=df[['reviewText','rating']]

In [5]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [6]:
# checking for missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [7]:
df.shape

(12000, 2)

In [8]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

### Preprocessing and Cleaning

In [9]:
# postive review is 0 and negative review is 1
df['rating']=df['rating'].apply(lambda x:1 if x<3 else 0)

In [10]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",0
1,Great short read. I didn't want to put it dow...,0
2,I'll start by saying this is the first of four...,0
3,Aggie is Angela Lansbury who carries pocketboo...,0
4,I did not expect this type of book to be in li...,0


In [11]:
df['rating'].unique()

array([0, 1], dtype=int64)

In [12]:
df['rating'].value_counts()

rating
0    8000
1    4000
Name: count, dtype: int64

In [13]:
# lower case
df['reviewText']=df['reviewText'].str.lower()

In [14]:
import re
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from bs4 import BeautifulSoup

In [16]:
# removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]+','',x))

#### Example:
```plaintext
                  reviewText                              cleanText
0          <p>This is <b>great</b>!</p>                 This is great!
1  <div>Awesome <a href='url'>product</a>!</div>       Awesome product!
2    <span>Highly <i>recommended</i>.</span>          Highly recommended.
```

In [17]:
df['reviewText']

0        jace rankin may be short but hes nothing to me...
1        great short read  i didnt want to put it down ...
2        ill start by saying this is the first of four ...
3        aggie is angela lansbury who carries pocketboo...
4        i did not expect this type of book to be in li...
                               ...                        
11995    valentine cupid is a vampire jena and ian anot...
11996    i have read all seven books in this series apo...
11997    this book really just wasnt my cuppa  the situ...
11998    tried to use it to charge my kindle it didnt e...
11999    taking instruction is a look into the often hi...
Name: reviewText, Length: 12000, dtype: object

In [18]:
# removing stopwords
stop_words=set(stopwords.words('english'))

In [19]:
df['reviewText']=df['reviewText'].apply(lambda x:' '.join([word for word in x.split()
                                                           if word not in stop_words]))

In [20]:
# remove URL
df['reviewText']

0        jace rankin may short hes nothing mess man hau...
1        great short read didnt want put read one sitti...
2        ill start saying first four books wasnt expect...
3        aggie angela lansbury carries pocketbooks inst...
4        expect type book library pleased find price right
                               ...                        
11995    valentine cupid vampire jena ian another vampi...
11996    read seven books series apocalypticadventure o...
11997    book really wasnt cuppa situation man capturin...
11998    tried use charge kindle didnt even register ch...
11999    taking instruction look often hidden world sex...
Name: reviewText, Length: 12000, dtype: object

In [21]:
df['reviewText']=df['reviewText'].apply(lambda x:
                 re.sub(r'(?:http|https|ftp|ssh)://[\w.-]+(?:\.[a-zA-Z]{2,})(?:[/?#]\S*)?','',str(x)))

### Optimized Regular Expression:
```regex
(?:http|https|ftp|ssh)://[\w.-]+(?:\.[a-zA-Z]{2,})(?:[/?#]\S*)?
```

---

### Breaking It Down with Examples:

#### 1. **Protocol Matching**: `(?:http|https|ftp|ssh)://`
- **Matches**:
  - `http://`
  - `https://`
  - `ftp://`
  - `ssh://`
- **Explanation**:
  - The group `(?: ... )` is **non-capturing**, meaning it does not save the matched content but ensures it matches one of the protocols (`http`, `https`, `ftp`, `ssh`).
  - Followed by `://`, which is a literal match of the standard URL separator.

---

#### 2. **Domain and Subdomain**: `[\w.-]+`
- **Matches**:
  - `example`
  - `sub.example`
  - `my-site.example123`
- **Explanation**:
  - `[\w.-]+`: Matches one or more characters that are:
    - `\w`: Alphanumeric characters (letters and digits) or underscores (`_`).
    - `.`: Periods (for subdomains).
    - `-`: Hyphens (for domain names).
- **Example URLs**:
  - `http://example.com`
  - `https://sub.domain.org`
  - `ftp://my-site.example123`

---

#### 3. **Top-Level Domain (TLD)**: `(?:\.[a-zA-Z]{2,})`
- **Matches**:
  - `.com`
  - `.org`
  - `.co.uk`
- **Explanation**:
  - `(?: ... )`: Non-capturing group to match the TLD.
  - `\.`: Matches a literal dot (`.`).
  - `[a-zA-Z]{2,}`: Matches a string of 2 or more alphabetic characters (e.g., `com`, `org`).
- **Example URLs**:
  - `http://example.com`
  - `https://domain.co.uk`
  - `ftp://site.org`

---

#### 4. **Optional Path, Query, or Fragment**: `(?:[/?#]\S*)?`
- **Matches**:
  - `/path/to/resource`
  - `?query=value`
  - `#fragment`
- **Explanation**:
  - `(?: ... )?`: Optional group to match the path, query, or fragment.
  - `[/?#]`: Matches a single character that starts a path (`/`), query (`?`), or fragment (`#`).
  - `\S*`: Matches zero or more non-whitespace characters (to capture everything after the separator).
- **Example URLs**:
  - `https://example.com/path/to/resource`
  - `http://example.com?query=value`
  - `https://example.com#fragment`


In [22]:
df['reviewText'].sample().values

array(['pretty good nice flow little one dimensional characters pile crap'],
      dtype=object)

In [23]:
# lemmatization
from nltk.stem import WordNetLemmatizer

In [24]:
lemma=WordNetLemmatizer()

In [25]:
df['reviewText']=df['reviewText'].apply(lambda x:' '.join(lemma.lemmatize(word) for word in x.split()))

In [26]:
X=df['reviewText']
y=df['rating']

### Train Test Split

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=.20,random_state=42)

### Bag Of Words

In [29]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [30]:
bow=CountVectorizer()

In [31]:
xtrain_bow=bow.fit_transform(xtrain).toarray()
xtest_bow=bow.transform(xtest).toarray()

# TF-IDF

In [32]:
tfidf=TfidfVectorizer()

In [33]:
xtrain_tfidf=tfidf.fit_transform(xtrain).toarray()
xtest_tfidf=tfidf.transform(xtest).toarray()

In [34]:
from sklearn.naive_bayes import GaussianNB

In [35]:
gnb_bow=GaussianNB().fit(xtrain_bow,ytrain)

In [36]:
gnb_tfidf=GaussianNB().fit(xtrain_tfidf,ytrain)

In [37]:
from sklearn.metrics import accuracy_score,classification_report

In [38]:
ypred_bow=gnb_bow.predict(xtest_bow)

In [39]:
ypred_tfidf=gnb_tfidf.predict(xtest_tfidf)

#### BOW metrics

In [40]:
accuracy_score(ytest,ypred_bow)

0.5775

In [41]:
print(classification_report(ytest,ypred_bow))

              precision    recall  f1-score   support

           0       0.74      0.56      0.64      1597
           1       0.41      0.62      0.50       803

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400



#### TFIDF - metrics

In [42]:
accuracy_score(ytest,ypred_tfidf)

0.57875

In [43]:
print(classification_report(ytest,ypred_tfidf))

              precision    recall  f1-score   support

           0       0.74      0.57      0.64      1597
           1       0.41      0.60      0.49       803

    accuracy                           0.58      2400
   macro avg       0.57      0.58      0.57      2400
weighted avg       0.63      0.58      0.59      2400



# Word2Vec

In [44]:
import gensim
from gensim.models import Word2Vec

In [45]:
df['reviewText'][0]

'jace rankin may short he nothing mess man hauled saloon undertaker know he famous bounty hunter oregon 1890s shot man saloon finished year long quest avenge sister murder trying figure next snottynosed farm boy rescued gang bully offer money kill man forced ranch reluctantly agrees bring man justice kill outright first need tell sister widower newskyla kyle springer bailey riding trail sleeping ground past month trying find jace want revenge man killed husband took ranch amongst crime shes keen detour jace want take realizes shes option hide behind boy persona best try keep pace confrontation along way get shot jace discovers kyles kyla come clean whole reason need scoundrel dead hope hell still help herthe book share touching moment slowblooming romance kyla find good reason fear men hide behind boy persona watching jace slowly pull shell help conquer fear endearing pain real deeplyrooted didnt disappear face sexiness neither understandable aversion marriage magically disappear round

In [46]:
df1=df.copy()

In [47]:
df1.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,0
1,great short read didnt want put read one sitti...,0
2,ill start saying first four book wasnt expecti...,0
3,aggie angela lansbury carry pocketbook instead...,0
4,expect type book library pleased find price right,0


In [48]:
words=[]
for i in range(len(df1)):
    review=df1['reviewText'][i].split()
    words.append(review)

In [49]:
words

[['jace',
  'rankin',
  'may',
  'short',
  'he',
  'nothing',
  'mess',
  'man',
  'hauled',
  'saloon',
  'undertaker',
  'know',
  'he',
  'famous',
  'bounty',
  'hunter',
  'oregon',
  '1890s',
  'shot',
  'man',
  'saloon',
  'finished',
  'year',
  'long',
  'quest',
  'avenge',
  'sister',
  'murder',
  'trying',
  'figure',
  'next',
  'snottynosed',
  'farm',
  'boy',
  'rescued',
  'gang',
  'bully',
  'offer',
  'money',
  'kill',
  'man',
  'forced',
  'ranch',
  'reluctantly',
  'agrees',
  'bring',
  'man',
  'justice',
  'kill',
  'outright',
  'first',
  'need',
  'tell',
  'sister',
  'widower',
  'newskyla',
  'kyle',
  'springer',
  'bailey',
  'riding',
  'trail',
  'sleeping',
  'ground',
  'past',
  'month',
  'trying',
  'find',
  'jace',
  'want',
  'revenge',
  'man',
  'killed',
  'husband',
  'took',
  'ranch',
  'amongst',
  'crime',
  'shes',
  'keen',
  'detour',
  'jace',
  'want',
  'take',
  'realizes',
  'shes',
  'option',
  'hide',
  'behind',
  'bo

In [50]:
model = Word2Vec(vector_size=100, window=5, min_count=5)

In [51]:
model.build_vocab(words)

In [52]:
model.train(words, total_examples=model.corpus_count, epochs=model.epochs)

(2757356, 3277155)

In [53]:
# To get all the vocabulary
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'much',
 'first',
 'well',
 'even',
 'didnt',
 'short',
 'know',
 'could',
 'way',
 'great',
 'make',
 'sex',
 'little',
 'dont',
 'thing',
 'want',
 'two',
 'think',
 'find',
 'plot',
 'also',
 'romance',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'take',
 'many',
 'woman',
 'lot',
 'written',
 'say',
 'kindle',
 'work',
 'thought',
 'bit',
 'found',
 'going',
 'give',
 'year',
 'interesting',
 'liked',
 'writing',
 'novel',
 'loved',
 'another',
 'feel',
 'got',
 'better',
 'come',
 'man',
 'hot',
 'still',
 'enough',
 'back',
 'though',
 'people',
 'reader',
 'made',
 'review',
 'something',
 'part',
 'friend',
 'star',
 'cant',
 'page',
 'need',
 'bad',
 'keep',
 'new',
 'wasnt',
 'free',
 'doesnt',
 'world',
 'relationship',
 'enjoy',
 'recommend',
 'together',
 'next',
 'start',
 'felt',
 'best',
 'put',

In [54]:
# vocabulary size
model.corpus_count

12000

In [55]:
model.wv.similar_by_word('good')

[('decent', 0.8657065033912659),
 ('great', 0.8350508213043213),
 ('easy', 0.8121255040168762),
 ('potential', 0.8084303140640259),
 ('satisfying', 0.8066027760505676),
 ('entertaining', 0.8020840287208557),
 ('ok', 0.8002272844314575),
 ('enjoyable', 0.7970961332321167),
 ('i8217ve', 0.7954894304275513),
 ('predictable', 0.7937658429145813)]

In [56]:
def avg_word2vec(doc):
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

In [57]:
X=[]
for i in range(len(words)):
    X.append(avg_word2vec(words[i]))

In [58]:
X

[array([-0.1057885 ,  0.3730538 ,  0.21058263,  0.08191919, -0.03088285,
        -0.48222294,  0.1337072 ,  0.6639518 , -0.20765905, -0.10360585,
        -0.04221861, -0.3380629 ,  0.06167629,  0.13747185, -0.01724206,
        -0.27473733,  0.07733415, -0.5144532 ,  0.15543474, -0.45795575,
         0.14083646,  0.08987889,  0.16490787, -0.06107148, -0.07280365,
        -0.02089486, -0.33522835, -0.04569903, -0.24535397,  0.109722  ,
         0.42987424, -0.13397087,  0.09648104, -0.30658036, -0.11820743,
         0.38167518,  0.02501774, -0.330248  , -0.16037801, -0.6634079 ,
        -0.00979001, -0.37770852, -0.04709684,  0.02526399,  0.03037788,
        -0.17340252,  0.0185216 , -0.03486294,  0.15130511,  0.205836  ,
         0.11400773, -0.19731814, -0.19002552,  0.09495594, -0.26035517,
         0.1705586 ,  0.17088771,  0.10364752, -0.32793605, -0.0650016 ,
         0.00287293,  0.09844466, -0.0590919 , -0.03607626, -0.30149293,
         0.35397062,  0.15361987,  0.35317734, -0.4

In [59]:
len(X)

12000

In [60]:
X=pd.DataFrame(X)

In [61]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=.20,random_state=42)

In [62]:
from sklearn.ensemble import RandomForestClassifier

In [63]:
rf=RandomForestClassifier()

In [64]:
rf.fit(xtrain,ytrain)

RandomForestClassifier()

In [65]:
ypred=rf.predict(xtest)

In [66]:
from sklearn.metrics import accuracy_score,classification_report

In [67]:
accuracy_score(ytest,ypred)

0.7758333333333334

In [68]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1597
           1       0.72      0.55      0.62       803

    accuracy                           0.78      2400
   macro avg       0.76      0.72      0.73      2400
weighted avg       0.77      0.78      0.77      2400

